<a href="https://colab.research.google.com/github/HerrySaraswat0708/Industrial-Training/blob/main/MovieRecommendationSystemML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
movies = pd.read_csv('/content/movies.csv')
rating = pd.read_csv('/content/ratings_small.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


**Data Preprocessing**

In [ ]:
#Removing the | symbol 
movies['genres']=movies['genres'].apply(lambda x:x.replace("|"," "))

In [ ]:
#Removing - between genres 
movies['genres']=movies['genres'].apply(lambda x:x.replace("-",""))

In [ ]:
#Converting string to list
movies['genres']=movies['genres'].apply(lambda x:x.split())

In [ ]:
#Deleting the duplicate movies
movies.drop_duplicates(subset=['title'],keep='first',inplace=True)

In [ ]:
#Setting the movieId columnas index
movies.set_index('movieId',inplace=True)

In [ ]:
#Creating vectors array for storing the genres informayion
import numpy as np
Genres = ['Action','Adventure','Animation','Children','Comedy','Crime','Documentry','Drama','Fantasy','Horror','Musical','Romance','SciFi','Suspense','Thriller','War']
vectors = np.empty([9056,16])
for i in range(0,9056):
  for j in range(0,16):
    gen=pd.DataFrame(movies.iloc[i]['genres'])
    vectors[i][j]=(gen[0].str.contains(Genres[j]).astype(int)).max()

In [ ]:
#Merging the movies and rating dataframe
data = movies.merge(rating,left_on='movieId',right_on='movieId')

In [ ]:
data.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",7,3.0,851866703
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",15,2.0,1134521380
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",5,4.0,1163374957
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",19,3.0,855192868
4,5,Father of the Bride Part II (1995),[Comedy],15,4.5,1093070098


In [ ]:
#Creating the pivot table
final_data=data.pivot(index='movieId',columns='userId',values='rating')

In [ ]:
final_data.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,Action,Adventure,Animation,Children,Comedy,Crime,Documentry,Drama,Fantasy,Horror,Musical,Romance,SciFi,Suspense,Thriller,War
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,3.0,3.5,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Adding genres columns in dataset
for i in range(0,16):
  final_data[Genres[i]]=vectors[:,i]

In [ ]:
final_data=final_data.fillna(0)

In [ ]:
data.drop_duplicates(subset=['movieId'],inplace=True)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.drop(columns=['index'],inplace=True)

In [ ]:
#Creating sparse matrix for our dataset
from scipy.sparse import csr_matrix
sparse_data = csr_matrix(final_data)

In [ ]:
#Creating and training the model
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=20,metric='cosine',algorithm='brute')
knn.fit(final_data)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [ ]:
#Making recommendation system function
def recommend(movie):
  movie_index = data[data['title'].str.contains(movie)].index[0]
  distances,indices=knn.kneighbors([final_data.iloc[movie_index]],n_neighbors=11)
  lst = []
  rec=sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x:x[1])[1:11]
  for i in data[data['title'].str.contains(movie)].index:
    if i== data[data['title'].str.contains(movie)].index[0]:
      pass
    else:
      lst.append(data.iloc[i].title)
  for i in rec:
    lst.append(data.iloc[i[0]].title)
  df = pd.DataFrame(lst)
  df.drop_duplicates(subset=[0],inplace=True)
  df.columns = ['Movies']
  return df
  

In [ ]:
#Checking our system
recommend("Pocahontas")

,Movies
0,Beauty and the Beast (1991)
1,Pinocchio (1940)
2,"Hunchback of Notre Dame, The (1996)"
3,Snow White and the Seven Dwarfs (1937)
4,Cinderella (1950)
5,"Little Mermaid, The (1989)"
6,"Lion King, The (1994)"
7,Mary Poppins (1964)
8,Fantasia (1940)
9,Dumbo (1941)
